In [1]:
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import os

In [2]:
path = os.getcwd()
pdbs = ['APD3', 'CancerPPD', 'DBBASP', 'FermFooDb', 'MAHMI', 'PepBDB', 'THPdb']

In [3]:
dfs = {}
for pdb in pdbs:
    dfs[pdb] = pd.read_csv(f'{path}/PDBs/{pdb}.csv')

In [4]:
count_id = 1
data = {
    'LBQCPDB ID': [],
    'Source ID': [],
    'Sequence': [],
    'Activity': [],
    'Source PDB': [],
    'DOI': []
}

In [5]:
# APD3
for i in tqdm(range(0, len(dfs['APD3']))):
    line = dfs['APD3'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['APD ID'])
    data['Sequence'].append(line['Sequence'])
    data['Activity'].append(line['Activity'])
    data['Source PDB'].append('APD3')
    data['DOI'].append(line['DOI'])
    count_id += 1

100%|██████████| 3579/3579 [00:00<00:00, 8017.78it/s]


In [6]:
# CancerPPD
for i in tqdm(range(0, len(dfs['CancerPPD']))):
    line = dfs['CancerPPD'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append('Not Found')
    data['Sequence'].append(line['SEQUENCE'])
    data['Activity'].append('Not Found')
    data['Source PDB'].append('CancerPPD')
    data['DOI'].append(line['DOI'])
    count_id += 1

100%|██████████| 5068/5068 [00:00<00:00, 8539.19it/s]


In [7]:
# DBBASP
for i in tqdm(range(0, len(dfs['DBBASP']))):
    line = dfs['DBBASP'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['ID'])
    data['Sequence'].append(line['SEQUENCE'])
    data['Activity'].append('Not Found')
    data['Source PDB'].append('DBBASP')
    data['DOI'].append(line['DOI'])
    count_id += 1

100%|██████████| 20420/20420 [00:02<00:00, 9106.91it/s]


In [8]:
# FermFooDb
for i in tqdm(range(0, len(dfs['FermFooDb']))):
    line = dfs['FermFooDb'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append('Not Found')
    data['Sequence'].append(line['Peptide Sequence'])
    data['Activity'].append(line['Activity'])
    data['Source PDB'].append('FermFooDb')
    data['DOI'].append(line['DOI'])
    count_id += 1

100%|██████████| 2205/2205 [00:00<00:00, 8286.69it/s]


In [9]:
# MAHMI
for i in tqdm(range(0, len(dfs['MAHMI']))):
    line = dfs['MAHMI'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['ID'])
    data['Sequence'].append(line['SEQUENCE'])
    data['Activity'].append(line['BIOACTIVITY'])
    data['Source PDB'].append('MAHMI')
    data['DOI'].append('Not Found')
    count_id += 1

100%|██████████| 276400/276400 [00:30<00:00, 9152.48it/s]


In [10]:
# PepBDB
for i in tqdm(range(0, len(dfs['PepBDB']))):
    line = dfs['PepBDB'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['Peptide ID'])
    data['Sequence'].append(line['Sequence'])
    data['Activity'].append('Not Found')
    data['Source PDB'].append('PepBDB')
    data['DOI'].append('Not Found')
    count_id += 1

100%|██████████| 13301/13301 [00:01<00:00, 8755.22it/s]


In [11]:
# THPdb
for i in tqdm(range(0, len(dfs['THPdb']))):
    line = dfs['THPdb'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['ID'])
    data['Sequence'].append(line['Sequence'])
    data['Activity'].append('Not Found')
    data['Source PDB'].append('THPdb')
    data['DOI'].append('Not Found')
    count_id += 1

100%|██████████| 852/852 [00:00<00:00, 8568.96it/s]


In [12]:
df = pd.DataFrame(data)

In [13]:
dt = datetime.now()
df.to_csv(f'{path}/PDBs/LBQCPDB_releases/LBQCPDB_{dt.strftime("%Y_%m_%d")}.csv', index=False)
df.to_csv(f'{path}/LBQCPDB.csv', index=False)

In [16]:
df.describe()

,LBQCPDB ID,Source ID,Sequence,Activity,Source PDB,DOI
count,321825,321825,321710,320682,321825,321825
unique,321825,313703,304878,407,7,5088
top,LBQCPDB_000001,Not Found,GIVEQCCTSICSLYQLENYCN,Immunomodulatory,MAHMI,Not Found
freq,1,7273,244,183169,276400,296008


In [17]:
df

,LBQCPDB ID,Source ID,Sequence,Activity,Source PDB,DOI
0,LBQCPDB_000001,AP00001,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV,"Anti-Gram+ & Gram-, Antifungal, candidacidal, ...",APD3,Not Found
1,LBQCPDB_000002,AP00002,YVPLPNVPQPGRRPFPTFPGQGPFNPKIKWPQGY,Anti-Gram+ & Gram-,APD3,Not Found
2,LBQCPDB_000003,AP00003,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...,Antifungal,APD3,Not Found
3,LBQCPDB_000004,AP00004,NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFDC,"Anti-Gram+, Antifungal",APD3,Not Found
4,LBQCPDB_000005,AP00005,VFIDILDKVENAIHNAAQVGIGFAKPFEKLINPK,Anti-Gram+,APD3,Not Found
...,...,...,...,...,...,...
321820,LBQCPDB_321821,1848,NaN,Not Found,THPdb,Not Found
321821,LBQCPDB_321822,1849,FPTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQ...,Not Found,THPdb,Not Found
321822,LBQCPDB_321823,1850,FPTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQ...,Not Found,THPdb,Not Found
321823,LBQCPDB_321824,1851,AIRRYYLGAVELSWDYRQSELLRELHVDTRFPATAPGALPLGPSVL...,Not Found,THPdb,Not Found
